In [0]:
print("hello world")

hello world


In [0]:
order_df=spark.read.csv('dbfs:/public/retail_db/orders',schema='order_id INT,order_date TIMESTAMP,order_customer_id INT,order_status STRING')

In [0]:
order_items_df = spark. \
    read. \
    csv(
        'dbfs:/public/retail_db/order_items',
        schema='''
            order_item_id INT, order_item_order_id INT, order_item_product_id INT, 
            order_item_quantity INT, order_item_subtotal FLOAT, order_item_product_price FLOAT
        '''
    )

In [0]:
import pyspark.sql.functions as f

In [0]:
daliy_revenue_df=order_df.\
    filter("order_status IN ('COMPLETE','CLOSED')").\
        join(order_items_df,order_df['order_id']==order_items_df['order_item_order_id']).\
            groupBy('order_id','order_date').\
                agg(f.round(f.sum('order_item_subtotal').alias('order_revenue')))

In [0]:
daliy_revenue_df.show()

+--------+-------------------+---------------------------------------------------+
|order_id|         order_date|round(sum(order_item_subtotal) AS order_revenue, 0)|
+--------+-------------------+---------------------------------------------------+
|     931|2013-07-30 00:00:00|                                             1030.0|
|    1081|2013-07-31 00:00:00|                                              880.0|
|    2247|2013-08-06 00:00:00|                                              800.0|
|    2602|2013-08-08 00:00:00|                                              730.0|
|    3017|2013-08-10 00:00:00|                                              800.0|
|    4367|2013-08-20 00:00:00|                                              630.0|
|    5090|2013-08-24 00:00:00|                                              740.0|
|    5950|2013-08-31 00:00:00|                                              430.0|
|    5968|2013-08-31 00:00:00|                                              300.0|
|   

In [0]:
daily_revenue_product_df=order_df.\
    filter("order_status IN ('COMPLETE','CLOSED')").\
        join(order_items_df,order_df['order_id']==order_items_df['order_item_order_id']).\
            groupBy('order_item_product_id','order_date').\
                agg(f.round(f.sum('order_item_subtotal')).alias('order_revenue'))

In [0]:
daily_revenue_product_df.show()

+---------------------+-------------------+-------------+
|order_item_product_id|         order_date|order_revenue|
+---------------------+-------------------+-------------+
|                  403|2013-08-04 00:00:00|       3120.0|
|                  926|2013-09-20 00:00:00|         48.0|
|                  771|2013-10-12 00:00:00|        160.0|
|                  906|2013-11-02 00:00:00|         75.0|
|                  172|2013-11-05 00:00:00|         60.0|
|                  564|2013-12-06 00:00:00|        300.0|
|                  276|2014-04-08 00:00:00|        128.0|
|                  191|2013-10-04 00:00:00|       5099.0|
|                  565|2013-10-11 00:00:00|         70.0|
|                  835|2013-10-20 00:00:00|         96.0|
|                  642|2013-11-17 00:00:00|        120.0|
|                  116|2013-11-23 00:00:00|        225.0|
|                  792|2013-11-27 00:00:00|         60.0|
|                  191|2013-12-16 00:00:00|        800.0|
|             

In [0]:

# Global Ranks - orderBy
# Ranks with in each partition or group - partitionBy and orderBy
# df.select(rank().over(Window.orderBy(col('revenue').desc())))
# df.select(rank().over(Window.partitionBy('order_date').orderBy(col('revenue').desc())))

In [0]:
display(daily_revenue_product_df.filter("order_date = '2014-01-01'").orderBy('order_date', f.col('order_revenue').desc()))

order_item_product_id,order_date,order_revenue
1004,2014-01-01T00:00:00Z,5600.0
191,2014-01-01T00:00:00Z,4400.0
365,2014-01-01T00:00:00Z,3839.0
502,2014-01-01T00:00:00Z,3300.0
957,2014-01-01T00:00:00Z,3300.0
1073,2014-01-01T00:00:00Z,2200.0
1014,2014-01-01T00:00:00Z,1999.0
403,2014-01-01T00:00:00Z,1820.0
627,2014-01-01T00:00:00Z,760.0
724,2014-01-01T00:00:00Z,500.0


In [0]:
from pyspark.sql.window import Window

In [0]:
#now use rank on to of the above query

display(daily_revenue_product_df.\
    filter("order_date = '2014-01-01'").\
        withColumn('drnk',f.dense_rank().\
            over(Window.orderBy(f.col('order_revenue').desc()))))
        #orderBy('order_date', f.col('order_revenue').desc())

order_item_product_id,order_date,order_revenue,drnk
1004,2014-01-01T00:00:00Z,5600.0,1
191,2014-01-01T00:00:00Z,4400.0,2
365,2014-01-01T00:00:00Z,3839.0,3
502,2014-01-01T00:00:00Z,3300.0,4
957,2014-01-01T00:00:00Z,3300.0,4
1073,2014-01-01T00:00:00Z,2200.0,5
1014,2014-01-01T00:00:00Z,1999.0,6
403,2014-01-01T00:00:00Z,1820.0,7
627,2014-01-01T00:00:00Z,760.0,8
724,2014-01-01T00:00:00Z,500.0,9


In [0]:
display(daily_revenue_product_df.\
    #filter("order_date = '2014-01-01'").\
        withColumn('drnk',f.dense_rank().\
            over(Window.partitionBy('order_date').orderBy(f.col('order_revenue').desc()))))

order_item_product_id,order_date,order_revenue,drnk
1004,2013-07-25T00:00:00Z,5600.0,1
191,2013-07-25T00:00:00Z,5099.0,2
957,2013-07-25T00:00:00Z,4500.0,3
365,2013-07-25T00:00:00Z,3359.0,4
1073,2013-07-25T00:00:00Z,3000.0,5
1014,2013-07-25T00:00:00Z,2799.0,6
403,2013-07-25T00:00:00Z,1950.0,7
502,2013-07-25T00:00:00Z,1650.0,8
627,2013-07-25T00:00:00Z,1080.0,9
226,2013-07-25T00:00:00Z,600.0,10


In [0]:
#we want top 5 in 2014 jan

display(daily_revenue_product_df.\
    filter("order_date = '2014-01-01'").\
        withColumn('drnk',f.dense_rank().\
            over(Window.orderBy(f.col('order_revenue').desc()))).\
                filter('drnk<=5'))

order_item_product_id,order_date,order_revenue,drnk
1004,2014-01-01T00:00:00Z,5600.0,1
191,2014-01-01T00:00:00Z,4400.0,2
365,2014-01-01T00:00:00Z,3839.0,3
502,2014-01-01T00:00:00Z,3300.0,4
957,2014-01-01T00:00:00Z,3300.0,4
1073,2014-01-01T00:00:00Z,2200.0,5


In [0]:
display(daily_revenue_product_df.\
    filter("order_date = '2014-01-01'").\
        withColumn('rnk',f.rank().\
            over(Window.orderBy(f.col('order_revenue').desc()))).\
                filter('rnk<=5'))

order_item_product_id,order_date,order_revenue,rnk
1004,2014-01-01T00:00:00Z,5600.0,1
191,2014-01-01T00:00:00Z,4400.0,2
365,2014-01-01T00:00:00Z,3839.0,3
502,2014-01-01T00:00:00Z,3300.0,4
957,2014-01-01T00:00:00Z,3300.0,4


In [0]:
#top 5 performing product in each date
spec=Window.partitionBy('order_date').orderBy(f.col('order_revenue').desc())

RANK Vs DENSE RANK

In [0]:
%sql
CREATE TABLE IF NOT EXISTS student_scores (
	student_id INT,
	student_score INT
);

In [0]:
%sql
INSERT OVERWRITE student_scores VALUES
(1, 980),
(2, 960),
(3, 960),
(4, 990),
(5, 920),
(6, 960),
(7, 980),
(8, 960),
(9, 940),
(10, 940);

num_affected_rows,num_inserted_rows
10,10


In [0]:
# display(
#     spark.read.table('student_scores').
#         orderBy(f.col('student_score').desc())
# )

student_scores=spark.read.table('student_scores').\
        orderBy(f.col('student_score').desc())

In [0]:
spec = Window.orderBy(f.col('student_score').desc())

In [0]:
display(
    student_scores. \
        withColumn('rnk', f.rank().over(spec)). \
        withColumn('drnk', f.dense_rank().over(spec)). \
        orderBy(f.col('student_score').desc())
)

student_id,student_score,rnk,drnk
4,990,1,1
1,980,2,2
7,980,2,2
2,960,4,3
3,960,4,3
6,960,4,3
8,960,4,3
9,940,8,4
10,940,8,4
5,920,10,5
